**Are California Police biased?**

Within the past few years, racial prejudices of police officers has been a hot topic. Partly to address this debate, provide transperancy, and provide some notion of formal analysis, Stanford has released the Open Policing Project which aggregates data from various states in the USA, consisting of approximately 60 million traffic stops. Since I am a California native, and California is well known for being a highly liberal state, I decided to analyze the California traffic stop data. This notebook in R consists of my findings and analysis. My blog probabilitymeasure.blog will contain commentary and interpretation of this notebook.

In [ ]:
#load packages
library(readr)
library(ggplot2)
library(zoo)
library(data.table)

In [ ]:
#load data
CA_cleaned <- read_csv("../input/CA_cleaned.csv", 
    col_types = cols(county_fips = col_skip(), 
        driver_age = col_skip(), driver_race_raw = col_skip(), 
        ethnicity = col_skip(), fine_grained_location = col_skip(), 
        id = col_skip(), location_raw = col_skip(), 
        police_department = col_skip(), search_type_raw = col_skip(), 
        state = col_skip(), stop_time = col_skip(), 
        violation_raw = col_skip()))

In [ ]:
dim(CA_cleaned)

As we can see the data is hyooge.

We have some missing data pertaining to the outcomes and the driver race, our variables of interest. The data is too large to try and impute it in a rigorous way (though we probably could), so for now we just remove it. Assuming that the missing data comes from the same distribution as our data set, the number of missing cases is inconsequential.

Later, we will take a closer look at the cases with missing contraband, but for now we set it aside.

In [ ]:
colMeans(is.na(CA_cleaned))

In [ ]:
columns <- c('driver_race', 'stop_outcome', 'is_arrested')
miss_cntrbnd <- CA_cleaned[is.na(CA_cleaned$contraband_found),]
data <- CA_cleaned[complete.cases(CA_cleaned[, columns]),]
data$contraband_missing <- (CA_cleaned$contraband_found == 'NA')
rm(CA_cleaned, columns)
sum(data$contraband_missing)

In [ ]:
colMeans(is.na(data))

In [ ]:
dim(data)
dim(miss_cntrbnd)

We don't lose too much.
The next block just recodes the variables to be slightly more useful. Take note of the `driver_age_raw` category. We won't be able to fit a model to the entire data without crashing, so we will look at subsets of the data. In particular, we will look at `driver_age_raw` at 2 and 3, which is the 15-25 and 25-32 age bracket, though almost all groups will get a look. We will also recode the violation and the dates. The missing values in `search_type` correspond to when a search was *not* conducted, so we can just add a string indicating that later. We also fill in the missing county name with a string instead of deleting those cases, as the missingness could be informative.

In [ ]:
data$driver_gender <- (data$driver_gender == 'M') * 1
data$search_conducted <- data$search_conducted * 1
data$is_arrested <- data$is_arrested * 1
data$contraband_found <- data$contraband_found * 1
data$stop_date <- as.Date(data$stop_date, '%Y-%m-%d')
data$stop_date <- as.yearmon(data$stop_date)
data$search_type[data$search_conducted == 1 & is.na(data$search_type)] <- 'None'

#looks like a logging error, we remove it
data <- data[!(data$driver_age_raw == '7'), ]

#relabel the rest
data$driver_age_raw[data$driver_age_raw == '0-14'] <- 1
data$driver_age_raw[data$driver_age_raw == '15-25'] <- 2
data$driver_age_raw[data$driver_age_raw == '25-32'] <- 3
data$driver_age_raw[data$driver_age_raw == '33-39'] <- 4
data$driver_age_raw[data$driver_age_raw == '40-48'] <- 5
data$driver_age_raw[data$driver_age_raw == '49+'] <- 6
data$driver_age_raw <- as.numeric(data$driver_age_raw)


data$violation[data$violation == 'DUI'] <- 0
data$violation[data$violation == 'Equipment'] <- 1
data$violation[data$violation == 'Moving violation'] <- 2
data$violation[data$violation == 'Other'] <- 3
data$violation <- as.numeric(data$violation)

data$county_name[is.na(data$county_name)] <- 'Missing'
data$count <- 1

#data$year <- year(data$stop_date)

In [ ]:
head(data)

**Description of the data**
The data we will work with contains the variables seen in the header above. Most of it is fairly straightforward. `stop_date` and `county_name` give the year and month of the stop. Presently, there is not strong reason to believe that days of the week or month will have a significant effect on stop outcomes (would you believe that Cops give more tickets on a Monday? Maybe, but do you believe it would cause statistically significant differences in practically census-level data?) and so we take the courser date-time to just be the month and year. This also eases plotting down below. Later we will further collapse that into year for the regression models unless a convincing reason is found/given to not do this. The time frame of the data ranges from 2009 to 2016. There are 55 counties in all, if the `"Missing"` county is included.

`search_conducted` is a binary indicator denoting whether a search was done, where 1 means "yes". We also have `search_type` which indicates how/why the search was conducted. Of particular note are the categories `"Probable Cause"` and `"Incident to Arrest"`.  

`contraband_found` is an indicator for whether some paraphilinilia was found. Unfortunately, we do not have data on what the contraband was, but it could range from (say) drugs to weapons.

`stop_outcome` is a categorical variable indicating what the end result of the traffic stop was. Of particular note is `"Verbal Warning"`, which we will analyze later. 

Finally, `is_arrested` is clear. It's an indicator whether that person was arrested or not. What is not clear, is why the arrest occurred. It is not true that the arrest always comes after a search, as some individuals were arrested, THEN searched as noted by `"Incident to Arrest"'. 

**Time Series Plotting**

We plot some time series trends in the data, grouped by race, after some aggregation to make the data actually readable. (If you try to plot this without aggregation, having thousands of stops on the same day for years makes the data too noisy to interpret).


In [ ]:
setDT(data)
time_data <- data[, lapply(.SD, sum, na.rm=TRUE),
           by=list(stop_date, driver_race),
                   .SDcols=c('search_conducted',
                             'is_arrested',
                             'contraband_found',
                             'contraband_missing',
                             'count')]
head(time_data)

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = time_data, aes(x = stop_date, y = search_conducted / count, colour = driver_race)) + 
       geom_line(size = 0.60) +
       ggtitle('Rates of Searches After Stop') +
       xlab('Stop Date') +
       ylab('Rate of Searches') +
       labs(colour = 'Driver Race')

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = time_data, aes(x = stop_date, y = is_arrested / count, colour = driver_race)) + 
       geom_line(size = 0.60) +
       ggtitle('Rate of Arrest After Stop') +
       xlab('Stop Date') +
       ylab('Arrest Rate') +
       labs(colour = 'Driver Race')

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = time_data,
       aes(x = stop_date,
           y = contraband_found / (search_conducted - contraband_missing),
           colour = driver_race)) + 
       geom_line(size = 0.60) +
       ggtitle('(Known) Contraband Hit rates') +
       xlab('Stop Date') +
       ylab('Hit Rate') +
       labs(colour = 'Driver Race')

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = time_data, aes(x = stop_date, y = count, colour = driver_race)) + 
       geom_line(size = 0.60) +
       ggtitle('Number of stops by Race') +
       xlab('Stop Date') +
       ylab('Number of Stops') +
       labs(colour = 'Driver Race')

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = time_data, aes(x = stop_date, y = search_conducted, colour = driver_race)) + 
       geom_line(size = 0.60) +
       ggtitle('Number of searches by Race') +
       xlab('Stop Date') +
       ylab('Number of Searches') +
       labs(colour = 'Driver Race')

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = time_data, aes(x = stop_date, y = contraband_found, colour = driver_race)) + 
       geom_line(size = 0.60) +
       ggtitle('Count of contraband by Race') +
       xlab('Stop Date') +
       ylab('(Known) Contraband Count') +
       labs(colour = 'Driver Race')

In [ ]:
sort( sapply(ls(),function(x){object.size(get(x))})) 

**Parameter estimates and simulations**

In [ ]:
#proportion of males in the data
mean(data$driver_gender)

In [ ]:
srch_rate <- mean(data$search_conducted)
wht_srch_rate <- mean(data[data$driver_race == 'White', ]$search_conducted)
his_srch_rate <- mean(data[data$driver_race == 'Hispanic', ]$search_conducted)
blk_srch_rate <- mean(data[data$driver_race == 'Black', ]$search_conducted)
asn_srch_rate <- mean(data[data$driver_race == 'Asian', ]$search_conducted)
otr_srch_rate <- mean(data[data$driver_race == 'Other', ]$search_conducted)

cat(sprintf('Search Rate (All): %f\n', srch_rate))
cat(sprintf('Search Rate (Whites): %f\n', wht_srch_rate))
cat(sprintf('Search Rate (Hispanics): %f\n', his_srch_rate))
cat(sprintf('Search Rate (Blacks): %f\n', blk_srch_rate))
cat(sprintf('Search Rate (Asians): %f\n', asn_srch_rate))
cat(sprintf('Search Rate (Other): %f\n', otr_srch_rate))

In [ ]:
n_stops <- dim(data)[1]
n_search <- sum(data$search_conducted)
n_hits <- sum(data$contraband_found, na.rm = TRUE)
srch_rate <- n_search / n_stops
hit_rate <- n_hits / n_search
#plot the posterior distribution
#prior is beta(1/2, 1/2)
srch_samples <- rbeta(n = 5000, shape1 = n_search + 1/2, shape2 = n_stops - n_search + 1/2)
hit_samples <- rbeta(n = 5000, shape1 = n_hits + 1/2, shape2 = n_search - n_hits + 1/2)

ggplot() + 
    geom_density(aes(x = srch_samples), 
                   alpha = 0.65, 
                   fill = 'blue') +
    xlab('Search Rate') +
    geom_vline(xintercept = srch_rate) +
    geom_text(x = srch_rate, label = srch_rate)

In [ ]:
ggplot() + 
    geom_density(aes(x = hit_samples), 
                   alpha = 0.45,
                   fill = 'red') +
    xlab('Hit Rate') +
    geom_vline(xintercept = hit_rate) +
    geom_text(x = hit_rate, label =  hit_rate)

In [ ]:
wht_n_stop <- dim(data[data$driver_race == 'White', ])[1]
wht_n_search <- sum(data[data$driver_race == 'White', ]$search_conducted)
wht_srch_rate <- wht_n_search / wht_n_stop

his_n_stop <- dim(data[data$driver_race == 'Hispanic', ])[1]
his_n_search <- sum(data[data$driver_race == 'Hispanic', ]$search_conducted)
his_srch_rate <- his_n_search / his_n_stop

blk_n_stop <- dim(data[data$driver_race == 'Black', ])[1]
blk_n_search <- sum(data[data$driver_race == 'Black', ]$search_conducted)
blk_srch_rate <- blk_n_search / blk_n_stop

asn_n_stop <- dim(data[data$driver_race == 'Asian', ])[1]
asn_n_search <- sum(data[data$driver_race == 'Asian', ]$search_conducted)
asn_srch_rate <- asn_n_search / asn_n_stop

otr_n_stop <- dim(data[data$driver_race == 'Other'])[1]
otr_n_search <- sum(data[data$driver_race == 'Other', ]$search_conducted)
otr_srch_rate <- otr_n_search / otr_n_stop

## MAKE AND PLOT PRIOR DISTRIBUTION OF SEARCH RATES FOR EACH RACE
## REPEAT THE SAME PROCESS IN ANOTHER BLOCK FOR THE HIT RATES

white_est <- data.frame(length = rbeta(n = 2000, 
                                       shape1 = wht_n_search + 1/2, 
                                       shape2 = wht_n_stop - wht_n_search + 1/2))
black_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = blk_n_search + 1/2,
                                          shape2 = blk_n_stop - blk_n_search + 1/2))

hispanic_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = his_n_search + 1/2,
                                          shape2 = his_n_stop - his_n_search + 1/2))

asian_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = asn_n_search + 1/2,
                                          shape2 = asn_n_stop - asn_n_search + 1/2))

other_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = otr_n_search + 1/2,
                                          shape2 = otr_n_stop - otr_n_search + 1/2))

white_est$race <- 'White'
black_est$race <- 'Black'
asian_est$race <- 'Asian'
hispanic_est$race <- 'Hispanic'
other_est$race <- 'Other'

sim_data <- rbind(white_est, 
                  black_est, 
                  asian_est, 
                  hispanic_est, 
                  other_est)

ggplot(sim_data, aes(length, fill = race)) + 
    geom_density(alpha = 0.60) +
    xlab('Posterior distribution for Pr(Search|Stop & Race)')

The following analyses will be done with complete case analysis

In [ ]:
data <- data[complete.cases(data[, 'contraband_found']),]

In [ ]:
wht_n_hits <- sum(data[data$driver_race == 'White' &
                             data$search_conducted == 1, ]$contraband_found)
wht_hit_rate <- wht_n_hits / wht_n_search

his_n_hits <- sum(data[data$driver_race == 'Hispanic' &
                             data$search_conducted == 1, ]$contraband_found)
his_hit_rate <- his_n_hits / his_n_search 

blk_n_hits <- sum(data[data$driver_race == 'Black' &
                             data$search_conducted == 1, ]$contraband_found)
blk_hit_rate <- blk_n_hits / blk_n_search

asn_n_hits <- sum(data[data$driver_race == 'Asian' &
                             data$search_conducted == 1, ]$contraband_found)
asn_hit_rate <- asn_n_hits / asn_n_search

otr_n_hits <- sum(data[data$driver_race == 'Other' & 
                             data$search_conducted == 1, ]$contraband_found)
otr_hit_rate <- otr_n_hits / otr_n_search 

## MAKE AND PLOT PRIOR DISTRIBUTION OF SEARCH RATES FOR EACH RACE
## REPEAT THE SAME PROCESS IN ANOTHER BLOCK FOR THE HIT RATES

white_est <- data.frame(length = rbeta(n = 2000, 
                                       shape1 = wht_n_hits + 1/2, 
                                       shape2 = wht_n_search - wht_n_hits + 1/2))
black_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = blk_n_hits + 1/2,
                                          shape2 = blk_n_search - blk_n_hits + 1/2))

hispanic_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = his_n_hits + 1/2,
                                          shape2 = his_n_search - his_n_hits + 1/2))

asian_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = asn_n_hits + 1/2,
                                          shape2 = asn_n_search - asn_n_hits + 1/2))

other_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = otr_n_hits + 1/2,
                                          shape2 = otr_n_search - otr_n_hits + 1/2))

white_est$race <- 'White'
black_est$race <- 'Black'
asian_est$race <- 'Asian'
hispanic_est$race <- 'Hispanic'
other_est$race <- 'Other'

sim_data <- rbind(white_est, 
                  black_est, 
                  asian_est, 
                  hispanic_est, 
                  other_est)

ggplot(sim_data, aes(length, fill = race)) + 
    geom_density(alpha = 0.60) +
    geom_vline(xintercept = wht_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = blk_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = asn_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = otr_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = his_hit_rate, linetype = 'dashed') +
    xlab('Posterior distribution for Pr(Contraband|Search & Race)')

In [ ]:
wht_n_search <- sum(data[data$driver_race == 'White' &
                               data$driver_age_raw == '2', ]$search_conducted)

wht_n_hits <- sum(data[data$driver_race == 'White' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2', ]$contraband_found)
wht_hit_rate <- wht_n_hits / wht_n_search

his_n_search <- sum(data[data$driver_race == 'Hispanic' &
                               data$driver_age_raw == '2', ]$search_conducted)

his_n_hits <- sum(data[data$driver_race == 'Hispanic' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2', ]$contraband_found)
his_hit_rate <- his_n_hits / his_n_search 

blk_n_search <- sum(data[data$driver_race == 'Black' &
                               data$driver_age_raw == '2', ]$search_conducted)

blk_n_hits <- sum(data[data$driver_race == 'Black' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2', ]$contraband_found)
blk_hit_rate <- blk_n_hits / blk_n_search

asn_n_search <- sum(data[data$driver_race == 'Asian' &
                               data$driver_age_raw == '2', ]$search_conducted)

asn_n_hits <- sum(data[data$driver_race == 'Asian' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2', ]$contraband_found)
asn_hit_rate <- asn_n_hits / asn_n_search

otr_n_search <- sum(data[data$driver_race == 'Other' & 
                               data$driver_age_raw == '2', ]$search_conducted)

otr_n_hits <- sum(data[data$driver_race == 'Other' & 
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2', ]$contraband_found)
otr_hit_rate <- otr_n_hits / otr_n_search 

## MAKE AND PLOT PRIOR DISTRIBUTION OF SEARCH RATES FOR EACH RACE
## REPEAT THE SAME PROCESS IN ANOTHER BLOCK FOR THE HIT RATES

white_est <- data.frame(length = rbeta(n = 2000, 
                                       shape1 = wht_n_hits + 1/2, 
                                       shape2 = wht_n_search - wht_n_hits + 1/2))
black_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = blk_n_hits + 1/2,
                                          shape2 = blk_n_search - blk_n_hits + 1/2))

hispanic_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = his_n_hits + 1/2,
                                          shape2 = his_n_search - his_n_hits + 1/2))

asian_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = asn_n_hits + 1/2,
                                          shape2 = asn_n_search - asn_n_hits + 1/2))

other_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = otr_n_hits + 1/2,
                                          shape2 = otr_n_search - otr_n_hits + 1/2))

white_est$race <- 'White'
black_est$race <- 'Black'
asian_est$race <- 'Asian'
hispanic_est$race <- 'Hispanic'
other_est$race <- 'Other'

sim_data <- rbind(white_est, 
                  black_est, 
                  asian_est, 
                  hispanic_est, 
                  other_est)

ggplot(sim_data, aes(length, fill = race)) + 
    geom_density(alpha = 0.60) +
    geom_vline(xintercept = wht_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = blk_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = asn_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = otr_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = his_hit_rate, linetype = 'dashed') +
    xlab('Posterior distribution for Pr(Contraband|Search & Race & 15-25 y/o)')

In [ ]:
wht_n_search <- sum(data[data$driver_race == 'White' &
                               data$driver_age_raw == '3', ]$search_conducted)

wht_n_hits <- sum(data[data$driver_race == 'White' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '3', ]$contraband_found)
wht_hit_rate <- wht_n_hits / wht_n_search

his_n_search <- sum(data[data$driver_race == 'Hispanic' &
                               data$driver_age_raw == '3', ]$search_conducted)

his_n_hits <- sum(data[data$driver_race == 'Hispanic' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '3', ]$contraband_found)
his_hit_rate <- his_n_hits / his_n_search 

blk_n_search <- sum(data[data$driver_race == 'Black' &
                               data$driver_age_raw == '3', ]$search_conducted)

blk_n_hits <- sum(data[data$driver_race == 'Black' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '3', ]$contraband_found)
blk_hit_rate <- blk_n_hits / blk_n_search

asn_n_search <- sum(data[data$driver_race == 'Asian' &
                               data$driver_age_raw == '3', ]$search_conducted)

asn_n_hits <- sum(data[data$driver_race == 'Asian' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '3', ]$contraband_found)
asn_hit_rate <- asn_n_hits / asn_n_search

otr_n_search <- sum(data[data$driver_race == 'Other' & 
                               data$driver_age_raw == '3', ]$search_conducted)

otr_n_hits <- sum(data[data$driver_race == 'Other' & 
                             data$search_conducted == 1 &
                             data$driver_age_raw == '3', ]$contraband_found)
otr_hit_rate <- otr_n_hits / otr_n_search 

## MAKE AND PLOT PRIOR DISTRIBUTION OF SEARCH RATES FOR EACH RACE
## REPEAT THE SAME PROCESS IN ANOTHER BLOCK FOR THE HIT RATES

white_est <- data.frame(length = rbeta(n = 2000, 
                                       shape1 = wht_n_hits + 1/2, 
                                       shape2 = wht_n_search - wht_n_hits + 1/2))
black_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = blk_n_hits + 1/2,
                                          shape2 = blk_n_search - blk_n_hits + 1/2))

hispanic_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = his_n_hits + 1/2,
                                          shape2 = his_n_search - his_n_hits + 1/2))

asian_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = asn_n_hits + 1/2,
                                          shape2 = asn_n_search - asn_n_hits + 1/2))

other_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = otr_n_hits + 1/2,
                                          shape2 = otr_n_search - otr_n_hits + 1/2))

white_est$race <- 'White'
black_est$race <- 'Black'
asian_est$race <- 'Asian'
hispanic_est$race <- 'Hispanic'
other_est$race <- 'Other'

sim_data <- rbind(white_est, 
                  black_est, 
                  asian_est, 
                  hispanic_est, 
                  other_est)

ggplot(sim_data, aes(length, fill = race)) + 
    geom_density(alpha = 0.60) +
    geom_vline(xintercept = wht_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = blk_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = asn_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = otr_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = his_hit_rate, linetype = 'dashed') +
    xlab('Posterior distribution for Pr(Contraband|Search & Race & 25-32 y/o)')

It is possible that the inclusion of individuals under suspicion of DUI may be adding a bias term to our estimates of the search and contraband rate. Suppose that a Police Officer pulls someone over for a DUI, and they actually are impaired. In this case, the officer essentially has probable cause on a silver platter, and will likely go forward with a search after arresting the driver. So, let's have a look at only those who have "Moving Violations". I.e., they were speeding, ran a light, or otherwise had an infraction that is fairly common on the road. We will stick to drivers who are 15-25, as they seem to carry contraband at higher rates.

In [ ]:
wht_n_search <- sum(data[data$driver_race == 'White' &
                               data$driver_age_raw == '2' &
                               violation == 2, ]$search_conducted)

wht_n_hits <- sum(data[data$driver_race == 'White' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2' &
                             violation == 2, ]$contraband_found)
wht_hit_rate <- wht_n_hits / wht_n_search

his_n_search <- sum(data[data$driver_race == 'Hispanic' &
                               data$driver_age_raw == '2' &
                               violation == 2, ]$search_conducted)

his_n_hits <- sum(data[data$driver_race == 'Hispanic' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2' &
                             violation ==2, ]$contraband_found)
his_hit_rate <- his_n_hits / his_n_search 

blk_n_search <- sum(data[data$driver_race == 'Black' &
                               data$driver_age_raw == '2' &
                               violation == 2, ]$search_conducted)

blk_n_hits <- sum(data[data$driver_race == 'Black' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2' &
                             violation == 2, ]$contraband_found)
blk_hit_rate <- blk_n_hits / blk_n_search

asn_n_search <- sum(data[data$driver_race == 'Asian' &
                               data$driver_age_raw == '2' &
                               violation == 2, ]$search_conducted)

asn_n_hits <- sum(data[data$driver_race == 'Asian' &
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2' &
                             violation == 2, ]$contraband_found)
asn_hit_rate <- asn_n_hits / asn_n_search

otr_n_search <- sum(data[data$driver_race == 'Other' & 
                               data$driver_age_raw == '2' &
                               violation == 2, ]$search_conducted)

otr_n_hits <- sum(data[data$driver_race == 'Other' & 
                             data$search_conducted == 1 &
                             data$driver_age_raw == '2' &
                             violation ==2, ]$contraband_found)
otr_hit_rate <- otr_n_hits / otr_n_search 

## MAKE AND PLOT PRIOR DISTRIBUTION OF SEARCH RATES FOR EACH RACE
## REPEAT THE SAME PROCESS IN ANOTHER BLOCK FOR THE HIT RATES

white_est <- data.frame(length = rbeta(n = 5000, 
                                       shape1 = wht_n_hits + 1/2, 
                                       shape2 = wht_n_search - wht_n_hits + 1/2))
black_est <- data.frame(length = rbeta(n = 5000, 
                                          shape1 = blk_n_hits + 1/2,
                                          shape2 = blk_n_search - blk_n_hits + 1/2))

hispanic_est <- data.frame(length = rbeta(n = 5000, 
                                          shape1 = his_n_hits + 1/2,
                                          shape2 = his_n_search - his_n_hits + 1/2))

asian_est <- data.frame(length = rbeta(n = 5000, 
                                          shape1 = asn_n_hits + 1/2,
                                          shape2 = asn_n_search - asn_n_hits + 1/2))

other_est <- data.frame(length = rbeta(n = 5000, 
                                          shape1 = otr_n_hits + 1/2,
                                          shape2 = otr_n_search - otr_n_hits + 1/2))

white_est$race <- 'White'
black_est$race <- 'Black'
asian_est$race <- 'Asian'
hispanic_est$race <- 'Hispanic'
other_est$race <- 'Other'

sim_data <- rbind(white_est, 
                  black_est, 
                  asian_est, 
                  hispanic_est, 
                  other_est)

ggplot(sim_data, aes(length, fill = race)) + 
    geom_density(alpha = 0.60) +
    geom_vline(xintercept = wht_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = blk_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = asn_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = otr_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = his_hit_rate, linetype = 'dashed') +
    xlab('Posterior distribution for Pr(Contraband|Search & Race & Age & Moving Violation)')

What if we only look at those individuals who were searched because of "probable cause"? This is an interesting one, because "probable cause" is essentially a judgement made by the officer. If the officer sees a certain amount of evidence of wrong doing, they then may decide that they have "probable cause". The long running joke here is the officer stating "I smell weed in your car", then conducting a search, when maurijuana may or may not actually be present.

More or less, these are searches that were made up to the officer's personal discretion, as opposed to a search conducted in the face of a confirmed DUI, which happens essentially automatically. 

In [ ]:
unique(data$search_type)
dim(data[data$search_type == 'Probable Cause',])

In [ ]:
wht_n_search <- sum(data[data$driver_race == 'White' &
                         data$search_type == 'Probable Cause', ]$search_conducted)

wht_n_hits <- sum(data[data$driver_race == 'White' &
                             data$search_conducted == 1 &
                             data$search_type == 'Probable Cause', ]$contraband_found)
wht_hit_rate <- wht_n_hits / wht_n_search

his_n_search <- sum(data[data$driver_race == 'Hispanic' &
                         data$search_type == 'Probable Cause', ]$search_conducted)

his_n_hits <- sum(data[data$driver_race == 'Hispanic' &
                       data$search_conducted == 1 &
                       data$search_type == 'Probable Cause', ]$contraband_found)
his_hit_rate <- his_n_hits / his_n_search 

blk_n_search <- sum(data[data$driver_race == 'Black' &
                         data$search_type == 'Probable Cause', ]$search_conducted)

blk_n_hits <- sum(data[data$driver_race == 'Black' &
                       data$search_conducted == 1 &
                       data$search_type == 'Probable Cause', ]$contraband_found)
blk_hit_rate <- blk_n_hits / blk_n_search

asn_n_search <- sum(data[data$driver_race == 'Asian' &
                         data$search_type == 'Probable Cause', ]$search_conducted)

asn_n_hits <- sum(data[data$driver_race == 'Asian' &
                       data$search_conducted == 1 &
                       data$search_type == 'Probable Cause', ]$contraband_found)
asn_hit_rate <- asn_n_hits / asn_n_search

otr_n_search <- sum(data[data$driver_race == 'Other' & 
                         data$search_type == 'Probable Cause', ]$search_conducted)

otr_n_hits <- sum(data[data$driver_race == 'Other' & 
                       data$search_conducted == 1 &
                       data$search_type == 'Probable Cause', ]$contraband_found)
otr_hit_rate <- otr_n_hits / otr_n_search 

## MAKE AND PLOT PRIOR DISTRIBUTION OF SEARCH RATES FOR EACH RACE
## REPEAT THE SAME PROCESS IN ANOTHER BLOCK FOR THE HIT RATES

white_est <- data.frame(length = rbeta(n = 2000, 
                                       shape1 = wht_n_hits + 1/2, 
                                       shape2 = wht_n_search - wht_n_hits + 1/2))
black_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = blk_n_hits + 1/2,
                                          shape2 = blk_n_search - blk_n_hits + 1/2))

hispanic_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = his_n_hits + 1/2,
                                          shape2 = his_n_search - his_n_hits + 1/2))

asian_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = asn_n_hits + 1/2,
                                          shape2 = asn_n_search - asn_n_hits + 1/2))

other_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = otr_n_hits + 1/2,
                                          shape2 = otr_n_search - otr_n_hits + 1/2))

white_est$race <- 'White'
black_est$race <- 'Black'
asian_est$race <- 'Asian'
hispanic_est$race <- 'Hispanic'
other_est$race <- 'Other'

sim_data <- rbind(white_est, 
                  black_est, 
                  asian_est, 
                  hispanic_est, 
                  other_est)

ggplot(sim_data, aes(length, fill = race)) + 
    geom_density(alpha = 0.60) +
    geom_vline(xintercept = wht_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = blk_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = asn_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = otr_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = his_hit_rate, linetype = 'dashed') +
    xlab('Posterior distribution for Pr(Contraband|Search & Race & Probable Cause)')

The next block is the arrest rate for those stopped for a DUI violation.

In [ ]:
srch_rate <- mean(data[data$violation == 0,]$is_arrested)
wht_srch_rate <- mean(data[data$driver_race == 'White' & data$violation == 0, ]$is_arrested)
his_srch_rate <- mean(data[data$driver_race == 'Hispanic' & data$violation == 0, ]$is_arrested)
blk_srch_rate <- mean(data[data$driver_race == 'Black' & data$violation == 0, ]$is_arrested)
asn_srch_rate <- mean(data[data$driver_race == 'Asian' & data$violation == 0, ]$is_arrested)
otr_srch_rate <- mean(data[data$driver_race == 'Other' & data$violation == 0, ]$is_arrested)

cat(sprintf('Search Rate (All): %f\n', srch_rate))
cat(sprintf('Search Rate (Whites): %f\n', wht_srch_rate))
cat(sprintf('Search Rate (Hispanics): %f\n', his_srch_rate))
cat(sprintf('Search Rate (Blacks): %f\n', blk_srch_rate))
cat(sprintf('Search Rate (Asians): %f\n', asn_srch_rate))
cat(sprintf('Search Rate (Other): %f\n', otr_srch_rate))

In [ ]:
srch_rate <- mean(data[data$violation != 0,]$search_conducted)
wht_srch_rate <- mean(data[data$driver_race == 'White' & data$violation != 0, ]$search_conducted)
his_srch_rate <- mean(data[data$driver_race == 'Hispanic' & data$violation != 0, ]$search_conducted)
blk_srch_rate <- mean(data[data$driver_race == 'Black' & data$violation != 0, ]$search_conducted)
asn_srch_rate <- mean(data[data$driver_race == 'Asian' & data$violation != 0, ]$search_conducted)
otr_srch_rate <- mean(data[data$driver_race == 'Other' & data$violation != 0, ]$search_conducted)

cat(sprintf('Search Rate (All): %f\n', srch_rate))
cat(sprintf('Search Rate (Whites): %f\n', wht_srch_rate))
cat(sprintf('Search Rate (Hispanics): %f\n', his_srch_rate))
cat(sprintf('Search Rate (Blacks): %f\n', blk_srch_rate))
cat(sprintf('Search Rate (Asians): %f\n', asn_srch_rate))
cat(sprintf('Search Rate (Other): %f\n', otr_srch_rate))

Now, what can we say about the missing contraband values? The first thing to notice is that every row that is missing a contraband value has `search_conducted = 1`, which is immediately suspicious. 

In [ ]:
sum(miss_cntrbnd$search_conducted)
dim(miss_cntrbnd)[1]

Is there some other difference between those with a missing contraband value and those with one?

In [ ]:
unique(data$search_type)

In [ ]:
unique(miss_cntrbnd$search_type)

In [ ]:
n_arrested <- sum(miss_cntrbnd$is_arrested)
n_stops <- dim(miss_cntrbnd)[1]
n_arrested / n_stops
n_arrested

So, it seems that those with missing contraband values have different values for the `search_type` field than those with values. Infact, these two sets are disjoint, and all the missing contraband values seem to correspond to a particular search_type. Moreover, about 63% of those with missing contraband values were arrested. Very suspicious so far. It certainly seems that we can't treat the data as missing at random.

What can we say about the race breakdown? What can we say about the arrest rates within the race break down?

In [ ]:
n_stops <- dim(miss_cntrbnd)[1]

n_whites <- dim(miss_cntrbnd[miss_cntrbnd$driver_race == 'White',])[1]
n_blacks <- dim(miss_cntrbnd[miss_cntrbnd$driver_race == 'Black',])[1]
n_hispanics <- dim(miss_cntrbnd[miss_cntrbnd$driver_race == 'Hispanic',])[1]
n_asians <- dim(miss_cntrbnd[miss_cntrbnd$driver_race == 'Asian',])[1]
n_others <- dim(miss_cntrbnd[miss_cntrbnd$driver_race == 'Other',])[1]

white_arrests <- sum(miss_cntrbnd[miss_cntrbnd$driver_race == 'White',]$is_arrested)
black_arrests <- sum(miss_cntrbnd[miss_cntrbnd$driver_race == 'Black',]$is_arrested)
hispanic_arrests <- sum(miss_cntrbnd[miss_cntrbnd$driver_race == 'Hispanic',]$is_arrested)
asian_arrests <- sum(miss_cntrbnd[miss_cntrbnd$driver_race == 'Asian',]$is_arrested)
other_arrests <- sum(miss_cntrbnd[miss_cntrbnd$driver_race == 'Other',]$is_arrested)

wht_arr_rate <- white_arrests / n_whites
blk_arr_rate <- black_arrests / n_blacks
his_arr_rate <- hispanic_arrests / n_hispanics
asn_arr_rate <- asian_arrests / n_asians
otr_arr_rate <- other_arrests / n_others

## MAKE AND PLOT PRIOR DISTRIBUTION OF SEARCH RATES FOR EACH RACE
## REPEAT THE SAME PROCESS IN ANOTHER BLOCK FOR THE HIT RATES

white_est <- data.frame(length = rbeta(n = 2000, 
                                       shape1 = white_arrests + 1/2, 
                                       shape2 = n_whites - white_arrests + 1/2))
black_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = black_arrests + 1/2,
                                          shape2 = n_blacks - black_arrests + 1/2))

hispanic_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = hispanic_arrests + 1/2,
                                          shape2 = n_hispanics - hispanic_arrests + 1/2))

asian_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = asian_arrests + 1/2,
                                          shape2 = n_asians - asian_arrests + 1/2))

other_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = other_arrests + 1/2,
                                          shape2 = n_others - other_arrests + 1/2))

white_est$race <- 'White'
black_est$race <- 'Black'
asian_est$race <- 'Asian'
hispanic_est$race <- 'Hispanic'
other_est$race <- 'Other'

sim_data <- rbind(white_est, 
                  black_est, 
                  asian_est, 
                  hispanic_est, 
                  other_est)

ggplot(sim_data, aes(length, fill = race)) + 
    geom_density(alpha = 0.60) +
    geom_vline(xintercept = wht_arr_rate, linetype = 'dashed') +
    geom_vline(xintercept = blk_arr_rate, linetype = 'dashed') +
    geom_vline(xintercept = asn_arr_rate, linetype = 'dashed') +
    geom_vline(xintercept = otr_arr_rate, linetype = 'dashed') +
    geom_vline(xintercept = his_arr_rate, linetype = 'dashed') +
    xlab('Posterior distribution for Arrest Probability')

In [ ]:


white_arrest / n_whites
black_arrest / n_blacks
hispanic_arrest / n_hispanics
asian_arrest / n_asians
other_arrest / n_others

So, this is still pretty interesting. When we stratify by race, the arrest rates are WAY higher for asians and whites than they are for blacks or hispanics, but everyone in this subset of data *was* searched. Why would this happen?

What would happen if we knew the data? How would our estimates and uncertainty change? We can try "imputing" the data with different values into our probability model from before and look at how those estimates change. We can try a couple different models.  First, let's suppose that everyone who has a missing contraband value actually *did* have contraband, and throw this into the model. Since everyone with a missing contraband value was searched, we can simply add the number of data points (by race) to our estimates from before.

In [ ]:
wht_n_hits <- sum(data[data$driver_race == 'White' &
                       data$search_conducted == 1, ]$contraband_found) + n_whites
wht_n_search <- sum(data[data$driver_race == 'White' &
                         data$search_conducted == 1, ]$search_conducted) + n_whites
wht_hit_rate <- wht_n_hits / wht_n_search



his_n_hits <- sum(data[data$driver_race == 'Hispanic' &
                             data$search_conducted == 1, ]$contraband_found) + n_hispanics
his_n_search <- sum(data[data$driver_race == 'Hispanic' &
                         data$search_conducted == 1, ]$search_conducted) + n_hispanics
his_hit_rate <- his_n_hits / his_n_search



blk_n_hits <- sum(data[data$driver_race == 'Black' &
                             data$search_conducted == 1, ]$contraband_found) + n_blacks
blk_n_search <- sum(data[data$driver_race == 'Black' &
                         data$search_conducted == 1, ]$search_conducted) + n_blacks
blk_hit_rate <- blk_n_hits / blk_n_search



asn_n_hits <- sum(data[data$driver_race == 'Asian' &
                             data$search_conducted == 1, ]$contraband_found) + n_asians
asn_n_search <- sum(data[data$driver_race == 'Asian' &
                         data$search_conducted == 1, ]$search_conducted) + n_asians
asn_hit_rate <- asn_n_hits / asn_n_search


otr_n_hits <- sum(data[data$driver_race == 'Other' & 
                             data$search_conducted == 1, ]$contraband_found) + n_others
otr_n_search <- sum(data[data$driver_race == 'Other' &
                         data$search_conducted == 1, ]$search_conducted) + n_others
otr_hit_rate <- otr_n_hits / otr_n_search

## MAKE AND PLOT PRIOR DISTRIBUTION OF SEARCH RATES FOR EACH RACE
## REPEAT THE SAME PROCESS IN ANOTHER BLOCK FOR THE HIT RATES

white_est <- data.frame(length = rbeta(n = 2000, 
                                       shape1 = wht_n_hits + 1/2, 
                                       shape2 = wht_n_search - wht_n_hits + 1/2))
black_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = blk_n_hits + 1/2,
                                          shape2 = blk_n_search - blk_n_hits + 1/2))

hispanic_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = his_n_hits + 1/2,
                                          shape2 = his_n_search - his_n_hits + 1/2))

asian_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = asn_n_hits + 1/2,
                                          shape2 = asn_n_search - asn_n_hits + 1/2))

other_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = otr_n_hits + 1/2,
                                          shape2 = otr_n_search - otr_n_hits + 1/2))

white_est$race <- 'White'
black_est$race <- 'Black'
asian_est$race <- 'Asian'
hispanic_est$race <- 'Hispanic'
other_est$race <- 'Other'

sim_data <- rbind(white_est, 
                  black_est, 
                  asian_est, 
                  hispanic_est, 
                  other_est)

ggplot(sim_data, aes(length, fill = race)) + 
    geom_density(alpha = 0.60) +
    geom_vline(xintercept = wht_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = blk_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = asn_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = otr_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = his_hit_rate, linetype = 'dashed') +
    xlab('Posterior distribution for Pr(Contraband|Search & Race)')

This is likely not too useful. We see that blacks would still carry contraband at a lower rate than whites, but the differences are pretty small, on the order of a 0.005 change in probability. However, there is little uncertainty in this change due to how many data points we have. While the difference is small, we could be relatively sure that it exists *if* our experiment were exactly true.

Suppose now that everyone who was initially missing their contraband was actually arrested *because* they had contraband. This is not totally unreasonable, as every one of those people were searched, and so the police likely had some suspicion they were carrying contraband. 

We can do this simply by adding the arrest number to the number of hits, and keeping everything else the same. Here, the number of arrests is serving as a proxy for contraband.

In [ ]:
wht_n_hits <- sum(data[data$driver_race == 'White' &
                       data$search_conducted == 1, ]$contraband_found) + white_arrests
wht_n_search <- sum(data[data$driver_race == 'White' &
                         data$search_conducted == 1, ]$search_conducted) + n_whites
wht_hit_rate <- wht_n_hits / wht_n_search



his_n_hits <- sum(data[data$driver_race == 'Hispanic' &
                             data$search_conducted == 1, ]$contraband_found) + hispanic_arrests
his_n_search <- sum(data[data$driver_race == 'Hispanic' &
                         data$search_conducted == 1, ]$search_conducted) + n_hispanics
his_hit_rate <- his_n_hits / his_n_search



blk_n_hits <- sum(data[data$driver_race == 'Black' &
                             data$search_conducted == 1, ]$contraband_found) + black_arrests
blk_n_search <- sum(data[data$driver_race == 'Black' &
                         data$search_conducted == 1, ]$search_conducted) + n_blacks
blk_hit_rate <- blk_n_hits / blk_n_search



asn_n_hits <- sum(data[data$driver_race == 'Asian' &
                             data$search_conducted == 1, ]$contraband_found) + asian_arrests
asn_n_search <- sum(data[data$driver_race == 'Asian' &
                         data$search_conducted == 1, ]$search_conducted) + n_asians
asn_hit_rate <- asn_n_hits / asn_n_search


otr_n_hits <- sum(data[data$driver_race == 'Other' & 
                             data$search_conducted == 1, ]$contraband_found) + other_arrests
otr_n_search <- sum(data[data$driver_race == 'Other' &
                         data$search_conducted == 1, ]$search_conducted) + n_others
otr_hit_rate <- otr_n_hits / otr_n_search

## MAKE AND PLOT PRIOR DISTRIBUTION OF SEARCH RATES FOR EACH RACE
## REPEAT THE SAME PROCESS IN ANOTHER BLOCK FOR THE HIT RATES

white_est <- data.frame(length = rbeta(n = 2000, 
                                       shape1 = wht_n_hits + 1/2, 
                                       shape2 = wht_n_search - wht_n_hits + 1/2))
black_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = blk_n_hits + 1/2,
                                          shape2 = blk_n_search - blk_n_hits + 1/2))

hispanic_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = his_n_hits + 1/2,
                                          shape2 = his_n_search - his_n_hits + 1/2))

asian_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = asn_n_hits + 1/2,
                                          shape2 = asn_n_search - asn_n_hits + 1/2))

other_est <- data.frame(length = rbeta(n = 2000, 
                                          shape1 = otr_n_hits + 1/2,
                                          shape2 = otr_n_search - otr_n_hits + 1/2))

white_est$race <- 'White'
black_est$race <- 'Black'
asian_est$race <- 'Asian'
hispanic_est$race <- 'Hispanic'
other_est$race <- 'Other'

sim_data <- rbind(white_est, 
                  black_est, 
                  asian_est, 
                  hispanic_est, 
                  other_est)

ggplot(sim_data, aes(length, fill = race)) + 
    geom_density(alpha = 0.60) +
    geom_vline(xintercept = wht_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = blk_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = asn_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = otr_hit_rate, linetype = 'dashed') +
    geom_vline(xintercept = his_hit_rate, linetype = 'dashed') +
    xlab('Posterior distribution for Pr(Contraband|Search & Race)')

It looks like now our estimates have a much larger difference, and in fact, Asians would have the highest probability of carrying contraband given that the police officer has decided to search them! In second place are Whites, with Hispanics and Blacks in last place. 


##TODO: formal random imputation of the missing values, conduct analysis then. Perhaps use propensity scores as well?